In [ ]:
!pip install colabcode


In [ ]:
from colabcode import  ColabCode
ColabCode(port=10000)

In [ ]:
!pip install transformers

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer
import torch

# initialize the model and tokenizer
tokenizer = BertTokenizer.from_pretrained('ProsusAI/finbert')
model = BertForSequenceClassification.from_pretrained('ProsusAI/finbert')

